In [4]:
# ================================================================================
# PREPROCESSING: ПОЛНЫЙ ЦИКЛ 
# ================================================================================

import pandas as pd
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder
import torch

print("="*80)
print("PREPROCESSING STARTED")
print("="*80)

drive_path = './'


PREPROCESSING STARTED


In [5]:

# ================================================================================
# 1. ЗАГРУЗКА И БАЗОВАЯ ОЧИСТКА
# ================================================================================

print("\n[1/9] Загрузка данных...")
ratings_df = pd.read_csv(f'{drive_path}/ratings.csv')
books_df = pd.read_csv(f'{drive_path}/books.csv')
tags_df = pd.read_csv(f'{drive_path}/tags.csv')
book_tags_df = pd.read_csv(f'{drive_path}/book_tags.csv')

print(f"   Загружено: ratings={len(ratings_df):,}, books={len(books_df):,}, tags={len(tags_df):,}, book_tags={len(book_tags_df):,}")

# Проверка корректности рейтингов
print("\n[2/9] Проверка рейтингов...")
invalid_ratings = (ratings_df['rating'] < 1) | (ratings_df['rating'] > 5)
fractional_ratings = (ratings_df['rating'] % 1 != 0)

if invalid_ratings.sum() > 0:
    print(f"    Найдено {invalid_ratings.sum()} некорректных рейтингов - удаляем")
    ratings_df = ratings_df[~invalid_ratings]

if fractional_ratings.sum() > 0:
    print(f"   Найдено {fractional_ratings.sum()} дробных рейтингов - округляем")
    ratings_df['rating'] = ratings_df['rating'].round().astype(int)
else:
    print(f"  Все рейтинги целые числа")

# Проверка дубликатов (но НЕ удаляем!)
duplicates = ratings_df.duplicated(subset=['user_id', 'book_id'], keep=False).sum()
print(f"   Найдено {duplicates} дубликатов (user_id, book_id) - оставляем все оценки")

print(f"  Итого ratings: {len(ratings_df):,} оценок")

# Очистка book_tags
neg_count = (book_tags_df['count'] < 0).sum()
if neg_count > 0:
    print(f"    Удалено {neg_count} записей с count < 0")
    book_tags_df = book_tags_df[book_tags_df['count'] > 0]



[1/9] Загрузка данных...
   Загружено: ratings=981,756, books=10,000, tags=34,252, book_tags=999,912

[2/9] Проверка рейтингов...
  Все рейтинги целые числа
   Найдено 4487 дубликатов (user_id, book_id) - оставляем все оценки
  Итого ratings: 981,756 оценок
    Удалено 6 записей с count < 0


In [6]:

# ================================================================================
# 2. ОБРАБОТКА ГОДА ПУБЛИКАЦИИ
# ================================================================================

print("\n[3/9] Обработка года публикации...")

# Статистика
print(f"   Книг с годом < 0: {(books_df['original_publication_year'] < 0).sum()}")
print(f"    Пропусков года: {books_df['original_publication_year'].isna().sum()}")

# Функция для категоризации эпох
def get_publication_era(year):
    """
    0 = Unknown
    1 = Ancient (< 0)
    2 = Medieval (0-1500)
    3 = Early Modern (1500-1800)
    4 = 19th century (1800-1900)
    5 = Early 20th (1900-1950)
    6 = Late 20th (1950-2000)
    7 = 21st century (2000+)
    """
    if pd.isna(year):
        return 0
    if year < 0:
        return 1
    if year < 1500:
        return 2
    if year < 1800:
        return 3
    if year < 1900:
        return 4
    if year < 1950:
        return 5
    if year < 2000:
        return 6
    return 7

# Создаем категориальный признак
books_df['publication_era'] = books_df['original_publication_year'].apply(get_publication_era)

# Нормализация года (сдвиг минимума к 0)
min_year = books_df['original_publication_year'].min()  # -1750
max_year = books_df['original_publication_year'].max()  # ~2025

print(f"    Диапазон годов: {min_year:.0f} - {max_year:.0f}")

# Нормализуем: сдвигаем так чтобы минимум стал 0
books_df['year_normalized'] = books_df['original_publication_year'] - min_year
books_df['year_normalized'] = books_df['year_normalized'].fillna(0).astype(int)

print(f"   Создано:")
print(f"     • publication_era (0-7): {books_df['publication_era'].value_counts().sort_index().to_dict()}")
print(f"     • year_normalized (0-{int(max_year - min_year)})")

# Удаляем исходный признак (чтобы не путаться с отрицательными годами)
books_df = books_df.drop(columns=['original_publication_year'])

# ================================================================================
# 3. ОБРАБОТКА ТЕКСТОВЫХ ПОЛЕЙ
# ================================================================================

print("\n[4/9] Обработка текстовых полей...")

# Проверяем реальные пропуски
print(f"    Пропуски:")
print(f"     • title: {books_df['title'].isna().sum()}")
print(f"     • authors: {books_df['authors'].isna().sum()}")
print(f"     • language_code: {books_df['language_code'].isna().sum()}")

# title - если есть пропуски
if books_df['title'].isna().sum() > 0:
    books_df['title'] = books_df['title'].fillna('Unknown Title')
books_df['title'] = books_df['title'].astype(str).str.strip()
books_df.loc[books_df['title'] == '', 'title'] = 'Unknown Title'

# authors - если есть пропуски
if books_df['authors'].isna().sum() > 0:
    books_df['authors'] = books_df['authors'].fillna('Unknown Author')
books_df['authors'] = books_df['authors'].astype(str).str.strip()
books_df.loc[books_df['authors'] == '', 'authors'] = 'Unknown Author'

# language_code - заполняем пропуски + КОДИРУЕМ (ИСПРАВЛЕНО!)
books_df['language_code'] = books_df['language_code'].fillna('eng').astype(str).str.strip()
books_df.loc[books_df['language_code'] == '', 'language_code'] = 'eng'

# Кодируем language_code в числа
from sklearn.preprocessing import LabelEncoder
le_lang = LabelEncoder()
books_df['language_code_encoded'] = le_lang.fit_transform(books_df['language_code'])

print(f"   Обработано: title, authors, language_code")
print(f"   Уникальных языков: {books_df['language_code'].nunique()}")
print(f"   language_code_encoded: диапазон 0-{books_df['language_code_encoded'].max()}")


# ================================================================================
# 4. СОЗДАНИЕ МАППИНГА ID
# ================================================================================

print("\n[5/9] Создание маппинга ID...")

# books_df['id'] = внутренний ID (1..10000) ← используется в ratings
# books_df['book_id'] = реальный Goodreads ID ← используется в book_tags

internal_to_goodreads = dict(zip(books_df['id'], books_df['book_id']))
goodreads_to_internal = dict(zip(books_df['book_id'], books_df['id']))

print(f"   Маппинг создан: {len(internal_to_goodreads):,} книг")

# ================================================================================
# 5. ОБРАБОТКА ТЕГОВ (БЕЗ ОГРАНИЧЕНИЯ ТОП-20)
# ================================================================================

print("\n[6/9] Обработка тегов...")

# Фильтруем мусорные теги
def is_valid_tag(tag):
    tag = str(tag).lower().strip()
    if len(tag) < 2 or len(tag) > 50:
        return False
    if tag.startswith(('-', '.', '#', '@')):
        return False
    # Удаляем теги с юникод-символами
    if not all(ord(c) < 128 for c in tag):
        return False
    # Удаляем чисто цифровые теги
    if tag.replace('-', '').isdigit():
        return False
    return True

tags_df_clean = tags_df[tags_df['tag_name'].apply(is_valid_tag)].copy()
print(f"   Чистых тегов: {len(tags_df_clean):,} / {len(tags_df):,}")

# Мержим book_tags с чистыми тегами
book_tags_clean = book_tags_df.merge(tags_df_clean, on='tag_id', how='inner')
print(f"   Записей book_tags после фильтрации: {len(book_tags_clean):,}")

# Агрегируем ВСЕ теги для каждой книги (сортируем по count, чтобы самые популярные были первыми)
book_tags_sorted = book_tags_clean.sort_values(['goodreads_book_id', 'count'], ascending=[True, False])

book_tags_aggregated = (
    book_tags_sorted
    .groupby('goodreads_book_id')['tag_name']
    .apply(lambda x: ' '.join(x.astype(str)))
    .reset_index()
)
book_tags_aggregated.columns = ['goodreads_book_id', 'tags_text']

print(f"  Агрегировано ВСЕХ тегов для {len(book_tags_aggregated):,} книг (БЕЗ ограничения топ-20)")

# Средняя длина тегов на книгу
avg_tags = book_tags_sorted.groupby('goodreads_book_id').size().mean()
print(f"   Среднее количество тегов на книгу: {avg_tags:.1f}")

# Конвертируем Goodreads ID → internal ID
book_tags_aggregated['internal_id'] = book_tags_aggregated['goodreads_book_id'].map(goodreads_to_internal)
book_tags_aggregated = book_tags_aggregated.dropna(subset=['internal_id'])
book_tags_aggregated['internal_id'] = book_tags_aggregated['internal_id'].astype(int)

print(f"   После маппинга: {len(book_tags_aggregated):,} книг")

# ================================================================================
# 6. МЕРДЖ В ЕДИНЫЙ ДАТАСЕТ (ДО SPLIT)
# ================================================================================

print("\n[7/9] Создание единого датасета...")

complete_df = ratings_df.copy()

# Добавляем метаданные книг
books_metadata = books_df[[
    'id', 'title', 'authors', 'average_rating', 'language_code_encoded',
    'year_normalized', 'publication_era', 'ratings_count', 'work_ratings_count'
]].rename(columns={'id': 'book_id'})

before_merge = len(complete_df)
complete_df = complete_df.merge(books_metadata, on='book_id', how='left')
lost_rows = before_merge - len(complete_df)
print(f"   Мердж с books: {before_merge:,} → {len(complete_df):,} (потеряно: {lost_rows})")

# Добавляем теги
book_tags_for_merge = book_tags_aggregated[['internal_id', 'tags_text']].rename(columns={'internal_id': 'book_id'})
before_merge = len(complete_df)
complete_df = complete_df.merge(book_tags_for_merge, on='book_id', how='left')
lost_rows = before_merge - len(complete_df)
print(f"   Мердж с tags: {before_merge:,} → {len(complete_df):,} (потеряно: {lost_rows})")

# Заполняем пропуски тегов
complete_df['tags_text'] = complete_df['tags_text'].fillna('')

print(f"   Complete dataset: {len(complete_df):,} строк × {len(complete_df.columns)} колонок")
print(f"   Уникальных книг: {complete_df['book_id'].nunique():,}")
print(f"   Уникальных пользователей: {complete_df['user_id'].nunique():,}")
print(f"   Книг с тегами: {(complete_df['tags_text'] != '').sum():,}")

# ================================================================================
# # ================================================================================
# 7. TRAIN/TEST SPLIT (ИСПРАВЛЕННАЯ ЛОГИКА + ОБРАБОТКА ДУБЛИКАТОВ)
# ================================================================================

print("\n[8/9] Train/Test split (temporal + stratified)...")

def stratified_temporal_split(df, test_size=3, min_interactions=5):
    """
    Для каждого пользователя с ≥5 взаимодействиями:
    - Последние test_size УНИКАЛЬНЫХ КНИГ → test
    - Остальное → train
    
    Для пользователей с <5 уникальных книг:
    - Всё → train
    
    Если юзер оценил книгу несколько раз - все оценки идут в один сплит
    """
    train_list, test_list = [], []
    holdout_dict = {}
    
    stats = {
        'users_total': 0,
        'users_in_test': 0,
        'users_only_train': 0,
        'train_size': 0,
        'test_size': 0,
        'duplicates_handled': 0
    }
    
    for user_id, group in df.groupby('user_id'):
        stats['users_total'] += 1
        
        # Группируем по book_id (чтобы все оценки одной книги шли вместе)
        book_groups = []
        for book_id, book_ratings in group.groupby('book_id'):
            if len(book_ratings) > 1:
                stats['duplicates_handled'] += len(book_ratings) - 1
            book_groups.append((book_id, book_ratings))
        
        # Проверяем количество УНИКАЛЬНЫХ книг
        n_unique_books = len(book_groups)
        
        if n_unique_books < min_interactions:
            # Мало уникальных книг → всё в train
            train_list.append(group)
            stats['train_size'] += len(group)
            stats['users_only_train'] += 1
            continue
        
        # Берем последние test_size КНИГ в test
        test_books = book_groups[-test_size:]
        train_books = book_groups[:-test_size]
        
        # Собираем датафреймы
        test_group = pd.concat([ratings for _, ratings in test_books], ignore_index=False)
        train_group = pd.concat([ratings for _, ratings in train_books], ignore_index=False)
        
        train_list.append(train_group)
        test_list.append(test_group)
        
        # В holdout сохраняем уникальные book_id
        holdout_dict[user_id] = [book_id for book_id, _ in test_books]
        
        stats['users_in_test'] += 1
        stats['train_size'] += len(train_group)
        stats['test_size'] += len(test_group)
    
    train_df = pd.concat(train_list, ignore_index=True)
    test_df = pd.concat(test_list, ignore_index=True) if test_list else pd.DataFrame()
    
    return train_df, test_df, holdout_dict, stats

train_df, test_df, holdout_dict, stats = stratified_temporal_split(complete_df)

print(f"   Train: {len(train_df):,} оценок")
print(f"   Test: {len(test_df):,} оценок")
print(f"   Пользователей в test: {stats['users_in_test']:,} / {stats['users_total']:,}")
print(f"    Пользователей только в train (< 5 уникальных книг): {stats['users_only_train']:,}")
print(f"    Дубликатов обработано: {stats['duplicates_handled']:,}")

# Проверка на data leakage
train_pairs = set(zip(train_df['user_id'], train_df['book_id']))
test_pairs = set(zip(test_df['user_id'], test_df['book_id']))
leakage = train_pairs & test_pairs

if len(leakage) > 0:
    print(f"   Data leakage: {len(leakage)} пар (user_id, book_id) в обоих сетах!")
    print(f"     Примеры: {list(leakage)[:5]}")
    raise ValueError("Data leakage detected!")
else:
    print(f"   Data leakage check: OK (нет пересечений по (user_id, book_id))")

# Проверка распределения рейтингов
print(f"    Распределение рейтингов:")
print(f"     Train: {train_df['rating'].value_counts().sort_index().to_dict()}")
print(f"     Test:  {test_df['rating'].value_counts().sort_index().to_dict()}")

# ================================================================================
# 8. СОЗДАНИЕ EMBEDDINGS 
# ================================================================================

print("\n[9/10] Создание embeddings...")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"  🖥️  Устройство: {device.upper()}")

model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
print(f"   Модель загружена (dim=384)")

# ============================================================================
# A) BOOK TAG EMBEDDINGS 
# ============================================================================

all_books = complete_df['book_id'].unique()
book_tags_available = book_tags_aggregated[book_tags_aggregated['internal_id'].isin(all_books)]

book_tags_list = book_tags_available['tags_text'].tolist()
book_ids_list = book_tags_available['internal_id'].tolist()

print(f"   Кодирование тегов для {len(book_tags_list):,} книг...")
book_tag_embeddings_array = model.encode(book_tags_list, batch_size=256, show_progress_bar=True)

book_tag_embeddings = {book_ids_list[i]: book_tag_embeddings_array[i] for i in range(len(book_ids_list))}

# Для книг без тегов - нулевой вектор
for book_id in all_books:
    if book_id not in book_tag_embeddings:
        book_tag_embeddings[book_id] = np.zeros(384)

print(f"   Book tag embeddings: {len(book_tag_embeddings):,} × 384")

# ============================================================================
# B) BOOK TEXT EMBEDDINGS (title + authors) 
# ============================================================================

print(f"   Кодирование title + authors для {len(books_df):,} книг...")

# Создаем текстовое представление: "title by authors"
books_df['text_for_embedding'] = books_df['title'] + ' by ' + books_df['authors']
book_text_list = books_df['text_for_embedding'].tolist()
book_ids_for_text = books_df['id'].tolist()  # internal ID

print(f"   Примеры текстов для эмбеддинга:")
print(f"     • {book_text_list[0][:80]}...")
print(f"     • {book_text_list[100][:80]}...")

book_text_embeddings_array = model.encode(book_text_list, batch_size=256, show_progress_bar=True)

book_text_embeddings = {book_ids_for_text[i]: book_text_embeddings_array[i] for i in range(len(book_ids_for_text))}

print(f"   Book text embeddings: {len(book_text_embeddings):,} × 384")

# ============================================================================
# C) КОМБИНИРОВАННЫЕ BOOK EMBEDDINGS
# ============================================================================

print(f"   Создание комбинированных book embeddings...")

# Для каждой книги: [tag_emb (384) + text_emb (384)] = 768
book_embeddings_combined = {}

for book_id in all_books:
    tag_emb = book_tag_embeddings.get(book_id, np.zeros(384))
    text_emb = book_text_embeddings.get(book_id, np.zeros(384))
    book_embeddings_combined[book_id] = np.concatenate([tag_emb, text_emb])

print(f"   Combined book embeddings: {len(book_embeddings_combined):,} × 768")

# ============================================================================
# D) USER EMBEDDINGS (на основе комбинированных book embeddings)
# ============================================================================

print(f"   User embeddings...")

def create_user_embeddings(df, book_emb_dict):
    """
    Создает user embeddings на основе взвешенного усреднения book embeddings
    """
    user_history = df.groupby('user_id', group_keys=False).apply(
        lambda x: list(zip(x['book_id'], x['rating'])), include_groups=False
    ).to_dict()
    
    # Определяем размерность эмбеддингов
    emb_dim = len(next(iter(book_emb_dict.values())))
    
    user_emb = {}
    for i, (user_id, books_ratings) in enumerate(user_history.items()):
        weighted_sum = np.zeros(emb_dim)
        weight_sum = 0
        
        for book_id, rating in books_ratings:
            if book_id in book_emb_dict:
                weight = (rating - 1) / 4.0  # нормализация [0, 1]
                weighted_sum += weight * book_emb_dict[book_id]
                weight_sum += weight
        
        user_emb[user_id] = weighted_sum / weight_sum if weight_sum > 0 else np.zeros(emb_dim)
        
        if (i + 1) % 10000 == 0:
            print(f"     {i + 1}/{len(user_history):,}...")
    
    return user_emb

print(f"   Train user embeddings...")
train_user_embeddings = create_user_embeddings(train_df, book_embeddings_combined)
print(f"   Train user embeddings: {len(train_user_embeddings):,} × 768")

print(f"   Test user embeddings...")
test_user_embeddings = create_user_embeddings(test_df, book_embeddings_combined)
print(f"   Test user embeddings: {len(test_user_embeddings):,} × 768")

# ================================================================================
# 9. ФИНАЛИЗАЦИЯ ДАТАСЕТОВ
# ================================================================================

print("\n[10/10] Финализация и сохранение...")

# Дропаем tags_text из train/test (оставляем только в complete)
train_final = train_df.drop(columns=['tags_text'])
test_final = test_df.drop(columns=['tags_text'])
complete_final = complete_df.copy()

print(f"   tags_text удален из train/test (оставлен в complete)")
print(f"   Train final: {train_final.shape}")
print(f"   Test final: {test_final.shape}")
print(f"   Complete: {complete_final.shape}")

# ================================================================================
# СОХРАНЕНИЕ
# ================================================================================

print("\nСохранение файлов...")

# CSV
print("  [1/8] complete_dataset.csv...")
complete_final.to_csv('complete_dataset.csv', index=False)

print("  [2/8] train_dataset.csv...")
train_final.to_csv('train_dataset.csv', index=False)

print("  [3/8] test_dataset.csv...")
test_final.to_csv('test_dataset.csv', index=False)

# PKL - embeddings 
print("  [4/8] train_embeddings.pkl...")
train_embeddings = {
    'book_emb': np.vstack([book_embeddings_combined[bid] for bid in train_final['book_id']]),
    'user_emb': np.vstack([train_user_embeddings[uid] for uid in train_final['user_id']]),
    'book_ids': train_final['book_id'].values,
    'user_ids': train_final['user_id'].values,
    'language_code': train_final['language_code_encoded'].values  
}
pickle.dump(train_embeddings, open('train_embeddings.pkl', 'wb'), protocol=4)

print("  [5/8] test_embeddings.pkl...")
test_embeddings = {
    'book_emb': np.vstack([book_embeddings_combined[bid] for bid in test_final['book_id']]),
    'user_emb': np.vstack([test_user_embeddings[uid] for uid in test_final['user_id']]),
    'book_ids': test_final['book_id'].values,
    'user_ids': test_final['user_id'].values,
    'language_code': test_final['language_code_encoded'].values  # НОВОЕ!
}
pickle.dump(test_embeddings, open('test_embeddings.pkl', 'wb'), protocol=4)

print("  [6/8] Словари embeddings...")
# Сохраняем ВСЕ типы эмбеддингов для гибкости
embeddings_dict = {
    'book_combined': book_embeddings_combined,      # 768-dim (tags + text)
    'book_tags': book_tag_embeddings,               # 384-dim (только теги)
    'book_text': book_text_embeddings               # 384-dim (только title+authors)
}
pickle.dump(embeddings_dict, open('book_embeddings.pkl', 'wb'), protocol=4)

pickle.dump(
    {'train': train_user_embeddings, 'test': test_user_embeddings},
    open('user_embeddings.pkl', 'wb'),
    protocol=4
)

print("  [7/8] holdout_dict.pkl...")
pickle.dump(holdout_dict, open('holdout_dict.pkl', 'wb'), protocol=4)

# Обновленные метаданные
print("  [8/8] preprocessing_metadata.pkl...")
metadata = {
    'n_ratings_total': len(complete_final),
    'n_ratings_train': len(train_final),
    'n_ratings_test': len(test_final),
    'n_users_total': complete_final['user_id'].nunique(),
    'n_users_train': len(train_user_embeddings),
    'n_users_test': len(test_user_embeddings),
    'n_books': len(book_embeddings_combined),
    'embedding_dims': {
        'book_tags': 384,
        'book_text': 384,
        'book_combined': 768,
        'user': 768,
        'language_code': 1
    },
    'publication_era_mapping': {
        0: 'Unknown',
        1: 'Ancient',
        2: 'Medieval',
        3: 'Early Modern',
        4: '19th century',
        5: 'Early 20th (1900-1950)',
        6: 'Late 20th (1950-2000)',
        7: '21st century'
    },
    'rating_distribution_train': train_final['rating'].value_counts().to_dict(),
    'rating_distribution_test': test_final['rating'].value_counts().to_dict(),
    'duplicates_kept': duplicates,
    'n_languages': books_df['language_code'].nunique()
}
pickle.dump(metadata, open('preprocessing_metadata.pkl', 'wb'), protocol=4)

print("\n" + "="*80)
print("PREPROCESSING ЗАВЕРШЕН")
print("="*80)
print(f"\nСоздано 8 файлов:")
print(f"  CSV:")
print(f"    • complete_dataset.csv: {complete_final.shape[0]:,} × {complete_final.shape[1]} (с tags_text)")
print(f"    • train_dataset.csv: {train_final.shape[0]:,} × {train_final.shape[1]} (без tags_text)")
print(f"    • test_dataset.csv: {test_final.shape[0]:,} × {test_final.shape[1]} (без tags_text)")
print(f"  PKL:")
print(f"    • train_embeddings.pkl: book {train_embeddings['book_emb'].shape} + user {train_embeddings['user_emb'].shape} + language")
print(f"    • test_embeddings.pkl: book {test_embeddings['book_emb'].shape} + user {test_embeddings['user_emb'].shape} + language")
print(f"    • book_embeddings.pkl: 3 типа (combined 768, tags 384, text 384)")
print(f"    • user_embeddings.pkl: train {len(train_user_embeddings):,} + test {len(test_user_embeddings):,} (768-dim)")
print(f"    • holdout_dict.pkl: {len(holdout_dict):,} пользователей")
print(f"    • preprocessing_metadata.pkl")





[3/9] Обработка года публикации...
   Книг с годом < 0: 31
    Пропусков года: 21
    Диапазон годов: -1750 - 2017
   Создано:
     • publication_era (0-7): {0: 21, 1: 31, 2: 19, 3: 75, 4: 254, 5: 466, 6: 2946, 7: 6188}
     • year_normalized (0-3767)

[4/9] Обработка текстовых полей...
    Пропуски:
     • title: 0
     • authors: 0
     • language_code: 1084
   Обработано: title, authors, language_code
   Уникальных языков: 25
   language_code_encoded: диапазон 0-24

[5/9] Создание маппинга ID...
   Маппинг создан: 10,000 книг

[6/9] Обработка тегов...
   Чистых тегов: 32,672 / 34,252
   Записей book_tags после фильтрации: 989,657
  Агрегировано ВСЕХ тегов для 10,000 книг (БЕЗ ограничения топ-20)
   Среднее количество тегов на книгу: 99.0
   После маппинга: 10,000 книг

[7/9] Создание единого датасета...
   Мердж с books: 981,756 → 981,756 (потеряно: 0)
   Мердж с tags: 981,756 → 981,756 (потеряно: 0)
   Complete dataset: 981,756 строк × 12 колонок
   Уникальных книг: 10,000
   Уник

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

   Book tag embeddings: 10,000 × 384
   Кодирование title + authors для 10,000 книг...
   Примеры текстов для эмбеддинга:
     • The Hunger Games (The Hunger Games, #1) by Suzanne Collins...
     • Me Talk Pretty One Day by David Sedaris...


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

   Book text embeddings: 10,000 × 384
   Создание комбинированных book embeddings...
   Combined book embeddings: 10,000 × 768
   User embeddings...
   Train user embeddings...
     10000/53,424...
     20000/53,424...
     30000/53,424...
     40000/53,424...
     50000/53,424...
   Train user embeddings: 53,424 × 768
   Test user embeddings...
     10000/35,659...
     20000/35,659...
     30000/35,659...
   Test user embeddings: 35,659 × 768

[10/10] Финализация и сохранение...
   tags_text удален из train/test (оставлен в complete)
   Train final: (874496, 11)
   Test final: (107260, 11)
   Complete: (981756, 12)

Сохранение файлов...
  [1/8] complete_dataset.csv...
  [2/8] train_dataset.csv...
  [3/8] test_dataset.csv...
  [4/8] train_embeddings.pkl...
  [5/8] test_embeddings.pkl...
  [6/8] Словари embeddings...
  [7/8] holdout_dict.pkl...
  [8/8] preprocessing_metadata.pkl...

PREPROCESSING ЗАВЕРШЕН

Создано 8 файлов:
  CSV:
    • complete_dataset.csv: 981,756 × 12 (с tags_text)
